## Data Cleaning and Preprocessing

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


### Initial Data Inspection

In [3]:
# Load the dataset
df = pd.read_csv('BTC-Daily.csv')

print("--- Initial Data Head ---")
print(df.head())
print("\n--- Data Info ---")
print(df.info())



--- Initial Data Head ---
         unix             date   symbol      open      high       low  \
0  1646092800  01/03/2022 0:00  BTC/USD  43221.71  43626.49  43185.48   
1  1646006400  28/02/2022 0:00  BTC/USD  37717.10  44256.08  37468.99   
2  1645920000  27/02/2022 0:00  BTC/USD  39146.66  39886.92  37015.74   
3  1645833600  26/02/2022 0:00  BTC/USD  39242.64  40330.99  38600.00   
4  1645747200  25/02/2022 0:00  BTC/USD  38360.93  39727.97  38027.61   

      close   Volume BTC    Volume USD  
0  43185.48    49.006289  2.116360e+06  
1  43178.98  3160.618070  1.364723e+08  
2  37712.68  1701.817043  6.418008e+07  
3  39146.66   912.724087  3.573010e+07  
4  39231.64  2202.851827  8.642149e+07  

--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2651 entries, 0 to 2650
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unix        2651 non-null   int64  
 1   date        2651 non-null   object 


### Data Preprocessing

In [4]:
# Convert 'date' to datetime objects
# format='%d/%m/%Y %H:%M' handles "01/03/2022 0:00"
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

# Sort by date (Oldest -> Newest) is CRITICAL for time-series
df = df.sort_values('date').reset_index(drop=True)

# Drop irrelevant columns
# 'unix' is redundant with date, 'symbol' is constant
df = df.drop(['unix', 'symbol'], axis=1)

display(df.head())

,date,open,high,low,close,Volume BTC,Volume USD
0,2014-11-28,363.59,381.34,360.57,376.28,3220878.18,8617.15
1,2014-11-29,376.42,386.60,372.25,376.72,2746157.05,7245.19
2,2014-11-30,376.57,381.99,373.32,373.34,1145566.61,3046.33
3,2014-12-01,376.40,382.31,373.03,378.39,2520662.37,6660.56
4,2014-12-02,378.39,382.86,375.23,379.25,2593576.46,6832.53


In [5]:
print("\n--- Data Preprocessing Completed ---")
display(df.head())


--- Data Preprocessing Completed ---


,date,open,high,low,close,Volume BTC,Volume USD
0,2014-11-28,363.59,381.34,360.57,376.28,3220878.18,8617.15
1,2014-11-29,376.42,386.60,372.25,376.72,2746157.05,7245.19
2,2014-11-30,376.57,381.99,373.32,373.34,1145566.61,3046.33
3,2014-12-01,376.40,382.31,373.03,378.39,2520662.37,6660.56
4,2014-12-02,378.39,382.86,375.23,379.25,2593576.46,6832.53


### Target Variable Derivation (The Bullish/Bearish Label)

In [6]:
# If Close(t+1) > Close(t) -> 1 (Bullish), else 0 (Bearish)
df['Target'] = (df['close'].shift(-1) > df['close']).astype(int)

print("\n--- Target Variable Created ---")
display(df[['date', 'close', 'Target']].head())


--- Target Variable Created ---


,date,close,Target
0,2014-11-28,376.28,1
1,2014-11-29,376.72,0
2,2014-11-30,373.34,1
3,2014-12-01,378.39,1
4,2014-12-02,379.25,0


### Feature Engineering

- Daily Return (% change)

In [7]:
df['Daily_Return'] = df['close'].pct_change() # 

- Volatility Features

In [8]:
df['High_Low_Spread'] = df['high'] - df['low'] # High-Low Spread (Intraday Volatility)
df['Close_Open_Spread'] = df['close'] - df['open'] # Close-Open Spread (Day's Momentum)

- Moving Averages (Trend Indicators)

In [9]:
df['MA_7'] = df['close'].rolling(window=7).mean()
df['MA_30'] = df['close'].rolling(window=30).mean()

# Drop rows with NaN values created by shifting/rolling (first 30 rows and last 1 row)
df = df.dropna()

### Save cleaned dataset

In [10]:
# Save the cleaned dataset to a new CSV file
output_path = "BTC_Cleaned_Data.csv"
df.to_csv(output_path, index=False)

print("Cleaned dataset saved to:", output_path)
display(df)

Cleaned dataset saved to: BTC_Cleaned_Data.csv


,date,open,high,low,close,Volume BTC,Volume USD,Target,Daily_Return,High_Low_Spread,Close_Open_Spread,MA_7,MA_30
29,2014-12-27,330.97,330.97,311.63,315.89,2.747753e+06,8.639540e+03,1,-0.045303,19.34,-15.08,325.351429,348.665333
30,2014-12-28,315.89,319.56,311.22,316.63,1.288907e+06,4.089610e+03,0,0.002343,8.34,0.74,324.420000,346.677000
31,2014-12-29,316.63,320.14,311.61,314.78,1.859592e+06,5.901060e+03,0,-0.005843,8.53,-1.85,322.171429,344.612333
32,2014-12-30,314.78,316.77,310.00,311.94,3.091942e+06,9.894240e+03,1,-0.009022,6.77,-2.84,318.807143,342.565667
33,2014-12-31,311.94,320.00,310.69,319.25,1.925335e+06,6.097710e+03,0,0.023434,9.31,7.31,318.394286,340.594333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2646,2022-02-25,38360.93,39727.97,38027.61,39231.64,2.202852e+03,8.642149e+07,0,0.022273,1700.36,870.71,38387.451429,40363.796333
2647,2022-02-26,39242.64,40330.99,38600.00,39146.66,9.127241e+02,3.573010e+07,0,-0.002166,1730.99,-95.98,38249.971429,40428.406333
2648,2022-02-27,39146.66,39886.92,37015.74,37712.68,1.701817e+03,6.418008e+07,1,-0.036631,2871.18,-1433.98,38155.511429,40427.217000
2649,2022-02-28,37717.10,44256.08,37468.99,43178.98,3.160618e+03,1.364723e+08,1,0.144946,6787.09,5461.88,39027.280000,40593.428000


### Normalization/Scaling

In [11]:
# # Initialize Min-Max Scaler to scale features between 0 and 1

# # Final list of features we will use for the model
# feature_cols = [
#     'open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD',
#     'Daily_Return', 'High_Low_Spread', 'Close_Open_Spread', 'MA_7', 'MA_30'
# ]

# print(f"\nFeatures Created: {len(feature_cols)}")
# print(f"Total rows ready for scaling: {len(df)}")


# scaler = MinMaxScaler(feature_range=(0, 1))

# df[feature_cols] = scaler.fit_transform(df[feature_cols])

# print("\n--- Data after Normalization (First 5 rows) ---")
# print(df[feature_cols].head())